# Environment setup

In [1]:
ls

Pipeline.ipynb  wine-quality-csv_pipeline.json  wine_quality_transformer.py
lab_test.ipynb  wine_quality_trainer.py


In [2]:
cd ../..

/tfx/src


In [3]:
rm -r /content

In [4]:
USERNAME = 'legall'
MAIL_PROVIDER = 'sfeir.com'

In [8]:
!gcloud auth login #(droits des comptes bq, gcs, vertex)

You are authorizing gcloud CLI without access to a web browser. Please run the following command on a machine with a web browser and copy its output back here. Make sure the installed gcloud version is 372.0.0 or newer.

gcloud auth login --remote-bootstrap="https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=q8HBI2Jkrm0KpCG71D8FFTi8FGi3mp&access_type=offline&code_challenge=Homip3qt4qzD83yi1xO8GORZ-jbgSqQaCtuqGl6khxo&code_challenge_method=S256&token_usage=remote"


Enter the output of the above command: ^C


Command killed by keyboard interrupt



In [ ]:
!git clone git@gitlab.com:Sfeir/sfeir-par/internals/data/devfest_mlops.git

In [5]:
GOOGLE_CLOUD_PROJECT = 'sfeir-data'
GOOGLE_CLOUD_REGION = 'europe-west1'
GCS_BUCKET_NAME = 'devfest_mlops_2022'

In [6]:
PIPELINE_NAME = 'wine-quality'

# Path to various pipeline artifact.
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}/{}'.format(
    GCS_BUCKET_NAME, USERNAME, PIPELINE_NAME)

# Paths for users' Python module.
MODULE_ROOT = 'gs://{}/pipeline_root/{}/{}'.format(
    GCS_BUCKET_NAME, USERNAME, PIPELINE_NAME)

ENDPOINT_NAME = 'prediction-' + PIPELINE_NAME + '-' + USERNAME

print('PIPELINE_ROOT: {}'.format(PIPELINE_ROOT))

PIPELINE_ROOT: gs://devfest_mlops_2022/pipeline_root/legall/wine-quality


In [7]:
QUERY = f"SELECT * FROM `{GOOGLE_CLOUD_PROJECT}.devfest_mlops.winequality_white_binary`"

In [8]:
_transformer_module_file = 'transformer.py'
_trainer_module_file = 'trainer.py'

# Pipelines elements

In [9]:
import os

import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import kfp
print('KFP version: {}'.format(kfp.__version__))

from monitorer_component import component

TensorFlow version: 2.9.0-rc2
TFX version: 1.9.1
KFP version: 1.8.14


In [10]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
context = InteractiveContext(pipeline_root='/content/interactive_pipeline/')

## Accessing data

In [11]:
output = tfx.proto.Output(
         split_config=tfx.proto.SplitConfig(splits=[
             tfx.proto.SplitConfig.Split(name='train', hash_buckets=4),
             tfx.proto.SplitConfig.Split(name='eval', hash_buckets=1)
         ]))

example_gen = tfx.components.CsvExampleGen(input_base="gs://devfest_mlops_2022/winequality_white_wine")

context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## Compiling statistics

In [12]:
statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [13]:
context.show(statistics_gen.outputs['statistics'])

## Extracting schema

In [14]:
schema_gen = tfx.components.SchemaGen(
      statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [15]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'alcohol',FLOAT,required,,-
'chlorides',FLOAT,required,,-
'citric_acid',FLOAT,required,,-
'density',FLOAT,required,,-
'fixed_acidity',FLOAT,required,,-
'free_sulfur_dioxide',FLOAT,required,,-
'ph',FLOAT,required,,-
'quality',INT,required,,-
'residual_sugar',FLOAT,required,,-


## Preparing the transformer (data cleaning, data transformation, feature engineering, )

In [16]:
transformer = tfx.components.Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_gen.outputs['schema'],
      module_file=os.path.join('devfest_mlops', 'src', _transformer_module_file))
context.run(transformer, enable_cache=False)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying create_pipeline.py -> build/lib
copying main.py -> build/lib
copying test_api_call.py -> build/lib
copying trainer.py -> build/lib
copying transformer.py -> build/lib
installing to /tmp/tmpqt3z3gug
running install
running install_lib
copying build/lib/transformer.py -> /tmp/tmpqt3z3gug
copying build/lib/test_api_call.py -> /tmp/tmpqt3z3gug
copying build/lib/create_pipeline.py -> /tmp/tmpqt3z3gug
copying build/lib/main.py -> /tmp/tmpqt3z3gug
copying build/lib/trainer.py -> /tmp/tmpqt3z3gug
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-inf

/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,


Processing /content/interactive_pipeline/_wheels/tfx_user_code_Transform-0.0+9e835449f36ee289e7f74de717cbfd50d4a12a994032b661265c96a7c3ba3108-py3-none-any.whl


Processing /content/interactive_pipeline/_wheels/tfx_user_code_Transform-0.0+9e835449f36ee289e7f74de717cbfd50d4a12a994032b661265c96a7c3ba3108-py3-none-any.whl


Processing /content/interactive_pipeline/_wheels/tfx_user_code_Transform-0.0+9e835449f36ee289e7f74de717cbfd50d4a12a994032b661265c96a7c3ba3108-py3-none-any.whl


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 4
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## Defining a training loop

In [17]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(os.path.join('devfest_mlops', 'src', _trainer_module_file)),
    examples=transformer.outputs['transformed_examples'],
    transform_graph=transformer.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'])
context.run(trainer, enable_cache=False)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying create_pipeline.py -> build/lib
copying main.py -> build/lib
copying test_api_call.py -> build/lib
copying trainer.py -> build/lib
copying transformer.py -> build/lib
installing to /tmp/tmpm21lltp0
running install
running install_lib
copying build/lib/transformer.py -> /tmp/tmpm21lltp0
copying build/lib/test_api_call.py -> /tmp/tmpm21lltp0
copying build/lib/create_pipeline.py -> /tmp/tmpm21lltp0
copying build/lib/main.py -> /tmp/tmpm21lltp0
copying build/lib/trainer.py -> /tmp/tmpm21lltp0
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.tx

/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,


writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpm21lltp0/tfx_user_code_Trainer-0.0+9e835449f36ee289e7f74de717cbfd50d4a12a994032b661265c96a7c3ba3108-py3.7.egg-info
running install_scripts
creating /tmp/tmpm21lltp0/tfx_user_code_Trainer-0.0+9e835449f36ee289e7f74de717cbfd50d4a12a994032b661265c96a7c3ba3108.dist-info/WHEEL
creating '/tmp/tmp7s9jqpyl/tfx_user_code_Trainer-0.0+9e835449f36ee289e7f74de717cbfd50d4a12a994032b661265c96a7c3ba3108-py3-none-any.whl' and adding '/tmp/tmpm21lltp0' to it
adding 'create_pipeline.py'
adding 'main.py'
adding 'test_api_call.py'
adding 'trainer.py'
adding 'transformer.py'
adding 'tfx_user_code_Trainer-0.0+9e835449f36ee289e7f74de717cbfd50d4a12a994032b661265c96a7c3ba3108.dist-info/METADATA'
adding 'tfx_user_code_Trainer-0.0+9e835449f36ee289e7f74de717cbfd50d4a12a994032b661265c96a7c3ba3108.dist-info/WHEEL'
adding 'tfx_user_code_Trainer-0.0+9e835449f36ee289e7f74de717cbfd50d4a12a994032b661265c96a

Epoch 1/200
320/320 [==============================] - 24s 68ms/step - loss: 0.4908 - accuracy: 0.7593 - val_loss: 0.5958 - val_accuracy: 0.6562
Epoch 2/200
320/320 [==============================] - 17s 50ms/step - loss: 0.4599 - accuracy: 0.7808 - val_loss: 0.5268 - val_accuracy: 0.7452
Epoch 3/200
320/320 [==============================] - 16s 49ms/step - loss: 0.4507 - accuracy: 0.7862 - val_loss: 0.5874 - val_accuracy: 0.7502
Epoch 4/200
320/320 [==============================] - 16s 49ms/step - loss: 0.4452 - accuracy: 0.7895 - val_loss: 0.5273 - val_accuracy: 0.7477
Epoch 5/200
320/320 [==============================] - 16s 49ms/step - loss: 0.4402 - accuracy: 0.7913 - val_loss: 0.5263 - val_accuracy: 0.7532
Epoch 6/200
320/320 [==============================] - 21s 65ms/step - loss: 0.4383 - accuracy: 0.7932 - val_loss: 0.5109 - val_accuracy: 0.7766
Epoch 7/200
320/320 [==============================] - 21s 65ms/step - loss: 0.4359 - accuracy: 0.7937 - val_loss: 0.5094 - val_ac

320/320 [==============================] - 16s 49ms/step - loss: 0.4125 - accuracy: 0.8088 - val_loss: 0.5436 - val_accuracy: 0.7477
Epoch 58/200
320/320 [==============================] - 16s 49ms/step - loss: 0.4125 - accuracy: 0.8091 - val_loss: 0.5579 - val_accuracy: 0.7551
Epoch 59/200
320/320 [==============================] - 21s 64ms/step - loss: 0.4128 - accuracy: 0.8085 - val_loss: 0.5406 - val_accuracy: 0.7366
Epoch 60/200
320/320 [==============================] - 21s 64ms/step - loss: 0.4105 - accuracy: 0.8094 - val_loss: 0.5439 - val_accuracy: 0.7477
Epoch 61/200
320/320 [==============================] - 21s 65ms/step - loss: 0.4128 - accuracy: 0.8084 - val_loss: 0.5478 - val_accuracy: 0.7348
Epoch 62/200
320/320 [==============================] - 16s 50ms/step - loss: 0.4100 - accuracy: 0.8103 - val_loss: 0.5390 - val_accuracy: 0.7538
Epoch 63/200
320/320 [==============================] - 16s 49ms/step - loss: 0.4119 - accuracy: 0.8101 - val_loss: 0.5428 - val_accuracy

...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: /content/interactive_pipeline/Trainer/model/5/Format-Serving/assets


INFO:tensorflow:Assets written to: /content/interactive_pipeline/Trainer/model/5/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 5
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

## Pusher

In [18]:
vertex_serving_spec = {
      'project_id': GOOGLE_CLOUD_PROJECT,
      'endpoint_name': ENDPOINT_NAME,
      'machine_type': 'n1-standard-4'
    }
serving_image = 'europe-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest'

pusher = tfx.extensions.google_cloud_ai_platform.Pusher(
  model=trainer.outputs['model'],
  custom_config={
      tfx.extensions.google_cloud_ai_platform.ENABLE_VERTEX_KEY:
          True,
      tfx.extensions.google_cloud_ai_platform.VERTEX_REGION_KEY:
          GOOGLE_CLOUD_REGION,
      tfx.extensions.google_cloud_ai_platform.VERTEX_CONTAINER_IMAGE_URI_KEY:
          serving_image,
      tfx.extensions.google_cloud_ai_platform.SERVING_ARGS_KEY:
        vertex_serving_spec,
  })
context.run(pusher)

Creating Endpoint


INFO:google.cloud.aiplatform.models:Creating Endpoint


Create Endpoint backing LRO: projects/881499317005/locations/europe-west1/endpoints/8880412369918885888/operations/4317190625020608512


INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/881499317005/locations/europe-west1/endpoints/8880412369918885888/operations/4317190625020608512


Endpoint created. Resource name: projects/881499317005/locations/europe-west1/endpoints/8880412369918885888


...

Deploy Endpoint model backing LRO: projects/881499317005/locations/europe-west1/endpoints/8880412369918885888/operations/4483823811233316864


INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/881499317005/locations/europe-west1/endpoints/8880412369918885888/operations/4483823811233316864


Endpoint model deployed. Resource name: projects/881499317005/locations/europe-west1/endpoints/8880412369918885888


INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/881499317005/locations/europe-west1/endpoints/8880412369918885888


ExecutionResult(
    component_id: Pusher
    execution_id: 6
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

### Shorter training loop for local testing

In [ ]:
serving_image = 'europe-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest'
vertex_serving_spec = {
      'project_id': GOOGLE_CLOUD_PROJECT,
      'endpoint_name': ENDPOINT_NAME,
      'machine_type': 'n1-standard-4'
}

pusher = tfx.extensions.google_cloud_ai_platform.Pusher(
    model=trainer.outputs['model'],
    custom_config={
        tfx.extensions.google_cloud_ai_platform.ENABLE_VERTEX_KEY:
            True,
        tfx.extensions.google_cloud_ai_platform.VERTEX_REGION_KEY:
            GOOGLE_CLOUD_REGION,
        tfx.extensions.google_cloud_ai_platform.VERTEX_CONTAINER_IMAGE_URI_KEY:
            serving_image,
        tfx.extensions.google_cloud_ai_platform.SERVING_ARGS_KEY:
          vertex_serving_spec,
    })
context.run(pusher)

## Monitorer

In [21]:
statistics = statistics_gen.outputs['statistics']
pushed_model = pusher.outputs['pushed_model']

pushed_model_destination_uri = pushed_model.get()[0].get_string_custom_property('pushed_destination')
features_file_uri = statistics.get()[0].uri
exec_properties = {'SAMPLE_RATE': 0, 'MONITORING_FREQUENCY': 3600, 'DEFAULT_THRESHOLD_VALUE': 0.03, 'EMAILS':""}

In [38]:
import json
import os
from typing import Any, Dict, List

from tfx import types
from tfx.dsl.components.base import base_executor
from tfx.dsl.io import fileio
from tfx.types import artifact_utils
from tfx.utils import io_utils

from google.cloud import aiplatform
import tensorflow_data_validation as tfdv
from google.cloud.aiplatform_v1.services.job_service import JobServiceClient
from google.cloud.aiplatform_v1.types import (SamplingStrategy, ModelMonitoringObjectiveConfig,
                                              ModelMonitoringAlertConfig, ThresholdConfig,
                                              ModelDeploymentMonitoringScheduleConfig,
                                              ModelDeploymentMonitoringObjectiveConfig,
                                              ModelDeploymentMonitoringJob,
                                              ListModelDeploymentMonitoringJobsRequest
                                              )
from google.protobuf.duration_pb2 import Duration


#pushed_model_destination_uri = input_dict['pushed_model'][0].get_string_custom_property('pushed_destination')
#features_file_uri = input_dict['statistics'][0].uri

pushed_destination_list = pushed_model_destination_uri.split('/')
project_id = pushed_destination_list[1]
region = pushed_destination_list[3]

aiplatform.init(project=project_id, location=region)

#endpoint = aiplatform.Endpoint.list(
  #filter=f"display_name={deployed_model_display_name}",
#  order_by="update_time"
#  )[-1]

model = aiplatform.Model(pushed_model_destination_uri)
#pusher_config = json.loads(pusher.exec_properties['custom_config'])

model_name=model.display_name
#endpoint.gca_resource.deployed_models[0].display_name
endpoint_id = model.to_dict()['deployedModels'][0]['endpoint']
#endpoint_id=endpoint.gca_resource.name.split('/')[-1]
deployed_model_id = model.to_dict()['deployedModels'][0]['deployedModelId']
#deployed_model_id=endpoint.list_models()[0].id


def get_features_names(features_file_uri: str):
    """
    Reads and parses json file containing features names from GCS bucket
    :param features_file_uri:
    :return:
    """
    def load_statistics(file_path: str):
      return tfdv.load_stats_binary(file_path)

    return load_statistics(features_file_uri+'/Split-train/FeatureStats.pb')

stats=get_features_names(features_file_uri)
api_vertex_endpoint = f"{region.upper()}-aiplatform.googleapis.com"

features = {}
for feature in stats.datasets[0].features:
  features[feature.path.step[0]]:{'mean':feature.num_stats.mean, 'std_dev':feature.num_stats.std_dev}
#numerical_features, categorical_features = get_features_names(features_file_uri)

sampling_config = SamplingStrategy.RandomSampleConfig(sample_rate=exec_properties['SAMPLE_RATE'])
sampling_strategy = SamplingStrategy(random_sample_config=sampling_config)

monitoring_duration = Duration(seconds=exec_properties['MONITORING_FREQUENCY'])
monitoring_config = ModelDeploymentMonitoringScheduleConfig(monitor_interval=monitoring_duration)

email_config = ModelMonitoringAlertConfig.EmailAlertConfig(user_emails=[exec_properties['EMAILS']])
alert_config = ModelMonitoringAlertConfig(email_alert_config=email_config)

# monitoring whether feature data distribution changes significantly over time
drift_thresholds = {}
default_threshold = ThresholdConfig(value=exec_properties['DEFAULT_THRESHOLD_VALUE'])

# set thresholds as default for all features
for feature in features:
    drift_thresholds[feature] = 1,96*features[feature]['std_dev']+features[feature]['mean']

drift_config = ModelMonitoringObjectiveConfig.PredictionDriftDetectionConfig(
    drift_thresholds=drift_thresholds
)

objective_config = ModelMonitoringObjectiveConfig(
    prediction_drift_detection_config=drift_config
)
monitoring_objective_configs = ModelDeploymentMonitoringObjectiveConfig(
    objective_config=objective_config
)
monitoring_objective_configs.deployed_model_id = deployed_model_id

# create the monitoring job
#endpoint = f"projects/{project_id}/locations/{region}/endpoints/{endpoint_id}"
predict_schema = ""
analysis_schema = ""

# monitoring job will  create up to 4 bq tables with names :
#   bq://<project_id>.model_deployment_monitoring_<endpoint_id>.<tolower(log_source)>_<tolower(log_type)>
# https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1beta1.types.ModelDeploymentMonitoringJob

parent = f"projects/{project_id}/locations/{region}"

monitoring_job_request = ListModelDeploymentMonitoringJobsRequest(
parent=parent, filter= f"display_name=monitoring_{model_name}"
)

options = dict(api_endpoint=api_vertex_endpoint)
client = JobServiceClient(client_options=options)
response = client.list_model_deployment_monitoring_jobs(
  request=monitoring_job_request)

if len([page for page in response.pages])==0:
    # response = client.update_model_deployment_monitoring_job(
    #  model_deployment_monitoring_job=monitoring_job)
    print("Monitoring job already active")
else:
    monitoring_job = ModelDeploymentMonitoringJob(
        display_name=f"monitoring_{model_name}",
        endpoint=endpoint_id,
        model_deployment_monitoring_objective_configs=[monitoring_objective_configs],
        logging_sampling_strategy=sampling_strategy,
        model_deployment_monitoring_schedule_config=monitoring_config,
        model_monitoring_alert_config=alert_config,
        predict_instance_schema_uri=predict_schema,
        analysis_instance_schema_uri=analysis_schema,
        enable_monitoring_pipeline_logs=True
    )
    response = client.create_model_deployment_monitoring_job(
      parent=parent, model_deployment_monitoring_job=monitoring_job
  )

In [37]:
if len([page for page in response.pages])==0:
    print('hello')
else:
    print('world')

world


In [45]:
monitoring_job

display_name: "monitoring_v1664726060"
endpoint: "projects/881499317005/locations/europe-west1/endpoints/3566164809621700608"
model_deployment_monitoring_objective_configs {
  deployed_model_id: "1304020790542336000"
  objective_config {
    prediction_drift_detection_config {
    }
  }
}
model_deployment_monitoring_schedule_config {
  monitor_interval {
    seconds: 3600
  }
}
logging_sampling_strategy {
  random_sample_config {
  }
}
model_monitoring_alert_config {
  email_alert_config {
    user_emails: ""
  }
}
enable_monitoring_pipeline_logs: true

In [49]:
model_name

'v1664726060'

In [48]:
f"display_name=monitoring_{model_name}"

'display_name=monitoring_v1664726060'

In [44]:
monitoring_job_request

parent: "projects/881499317005/locations/europe-west1"
filter: "display_name=monitoring_v1664726060"

In [ ]:
v1664542766

In [47]:
response = client.create_model_deployment_monitoring_job(
      parent=parent, model_deployment_monitoring_job=monitoring_job
  )
response

FailedPrecondition: 400 ModelMonitoring config [endpoint: 3566164809621700608
model_monitoring_schedule_config {
  monitoring_interval {
    seconds: 3600
  }
  monitoring_window {
    seconds: 3600
  }
}
logging_sampling_strategy {
  random_sample_config {
  }
}
model_monitoring_alert_config {
  email_alert_config {
    user_emails: ""
  }
}
bigquery_tables {
  log_source: SERVING
  log_type: PREDICT
  bigquery_table_path: "bq://sfeir-data.model_deployment_monitoring_3566164809621700608.serving_predict"
}
is_enabled: true
model_monitoring_objective_configs {
  prediction_drift_detection_config {
  }
  deployed_model_id: "1304020790542336000"
}
predict_instance_schema_uri: "gs://cloud-ai-platform-530638ec-999a-48f7-b673-5edfd62815f2/instance_schemas/job-8563186772631093248/predict"
predict_instance_schema {
  type_url: "type.googleapis.com/cloud.ai.platform.openapi.Schema"
}
tfdv_instance_schema_uri: "gs://cloud-ai-platform-530638ec-999a-48f7-b673-5edfd62815f2/instance_schemas/job-8563

In [27]:
monitoring = component.MonitorerComponent(
    statistics=statistics_gen.outputs['statistics'],
    pushed_model=pusher.outputs['pushed_model'],
)

context.run(monitoring)

Monitoring job already active


ExecutionResult(
    component_id: MonitorerComponent
    execution_id: 7
    outputs:
)

## Cleaning up

In [ ]:
pouyo

# Wrapping up everything

In [40]:
BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS = [
   '--project=' + GOOGLE_CLOUD_PROJECT,
   '--temp_location=' + os.path.join('gs://', GCS_BUCKET_NAME, 'tmp'),
   ]

PIPELINE_DEFINITION_FILE = PIPELINE_NAME + '_pipeline.json'

pipeline_name=PIPELINE_NAME
pipeline_root=PIPELINE_ROOT
query=QUERY
transformer_module_file=os.path.join(MODULE_ROOT, _transformer_module_file)
trainer_module_file=os.path.join(MODULE_ROOT, _trainer_module_file)
beam_pipeline_args=BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS

NameError: name '_transformer_module_file' is not defined

In [ ]:
#!gsutil cp {_transformer_module_file} {MODULE_ROOT}/
#!gsutil cp {_trainer_module_file} {MODULE_ROOT}/

In [ ]:
from typing import List, Optional

def _create_pipeline(pipeline_name: str, pipeline_root: str, query: str,
                     transformer_module_file:str,
                     trainer_module_file: str,
                     endpoint_name: str,
                     project_id: str, region: str,
                     beam_pipeline_args: Optional[List[str]],
                     ) -> tfx.dsl.Pipeline:
  """Creates a TFX pipeline using BigQuery."""

  # query data in BigQuery as a data source.
  output = tfx.proto.Output(
             split_config=tfx.proto.SplitConfig(splits=[
                 tfx.proto.SplitConfig.Split(name='train', hash_buckets=4),
                 tfx.proto.SplitConfig.Split(name='eval', hash_buckets=1)
             ]))
  
  example_gen = tfx.extensions.google_cloud_big_query.BigQueryExampleGen(
      query=query, output_config=output)
  
  # compute the statistics
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])
  
  # generate schema
  schema_gen = tfx.components.SchemaGen(
      statistics=statistics_gen.outputs['statistics'])
  
  # pre-processe data
  transformer = tfx.components.Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_gen.outputs['schema'],
      module_file=transformer_module_file)

  # use user-provided Python function that trains a model
  trainer = tfx.components.Trainer(
      module_file=trainer_module_file,
      examples=transformer.outputs['transformed_examples'],
      transform_graph=transformer.outputs['transform_graph'],
      schema=schema_gen.outputs['schema'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # pushe the model to model registry
  vertex_serving_spec = {
      'project_id': project_id,
      'endpoint_name': endpoint_name,
      'machine_type': 'n1-standard-4'
  }
  serving_image = 'europe-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest'

  pusher = tfx.extensions.google_cloud_ai_platform.Pusher(
      model=trainer.outputs['model'],
      custom_config={
          tfx.extensions.google_cloud_ai_platform.ENABLE_VERTEX_KEY:
              True,
          tfx.extensions.google_cloud_ai_platform.VERTEX_REGION_KEY:
              region,
          tfx.extensions.google_cloud_ai_platform.VERTEX_CONTAINER_IMAGE_URI_KEY:
              serving_image,
          tfx.extensions.google_cloud_ai_platform.SERVING_ARGS_KEY:
            vertex_serving_spec,
      })

  components = [
      example_gen,
      statistics_gen,
      schema_gen,
      transformer,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=components,
      beam_pipeline_args=beam_pipeline_args)

In [ ]:
import os

BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS = [
   '--project=' + GOOGLE_CLOUD_PROJECT,
   '--temp_location=' + os.path.join('gs://', GCS_BUCKET_NAME, 'tmp'),
   ]

PIPELINE_DEFINITION_FILE = PIPELINE_NAME + '_pipeline.json'
custom_tfx_image = 'europe-docker.pkg.dev/sfeir-data/eu.gcr.io/tfx_augm:1.9.1'

runner = tfx.orchestration.experimental.KubeflowV2DagRunner(
    config=tfx.orchestration.experimental.KubeflowV2DagRunnerConfig(default_image=custom_tfx_image),
    output_filename=PIPELINE_DEFINITION_FILE)
_ = runner.run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        query=QUERY,
        transformer_module_file=os.path.join(MODULE_ROOT, _transformer_module_file),
        trainer_module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
        endpoint_name=ENDPOINT_NAME,
        project_id=GOOGLE_CLOUD_PROJECT,
        region=GOOGLE_CLOUD_REGION,
        beam_pipeline_args=BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS))

In [ ]:
_

# Executing the job on Vertex

In [ ]:
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
import logging
logging.getLogger().setLevel(logging.INFO)

aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=GOOGLE_CLOUD_REGION)

job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=PIPELINE_NAME)
job.submit()